# Defining final simulation workflow

## Imports

In [1]:
# Supressing annoying warnings (!must be done first!)
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

# Logging
from rich.progress import Progress, track
from polysaccharide2.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    # force=True
)
LOGGER = logging.Logger(__name__)

from typing import Optional, Union
from pathlib import Path

/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


## Global parameters

In [2]:
mol_name = 'peg_modified'

pdb_sub = 'simple_polymers'
pdb_src_dir  = Path(f'pdb_test_cleaned/pdbs/{pdb_sub}')
mono_src_dir = Path(f'pdb_test_cleaned/monos/{pdb_sub}')

working_dir = Path(f'workflow_test_{mol_name}')
working_dir.mkdir(exist_ok=True)

## 0) Filetree setup

In [3]:
from shutil import copyfile
from polysaccharide2.genutils.fileutils.pathutils import assemble_path
from polysaccharide2.monomers import MonomerGroup

term_orients = { 
    'peg_modified' : {
        'peg_TERM2' : 'head',
        'peg_TERM3' : 'tail',
    },
    'paam_modified' : {
        'paam_TERM2' : 'head',
        'paam_TERM3' : 'tail',
    },
    'pnipam_modified' : {
        'pnipam_TERM2' : 'head',
        'pnipam_TERM3' : 'tail',
    },
}

starting_dir = working_dir / 'chemistry_seeds'
starting_dir.mkdir(exist_ok=True)

# COPY PDB FILE
pdb_src_path = assemble_path(pdb_src_dir, mol_name, extension='pdb')
assert(pdb_src_path.exists())
pdb_path = assemble_path(starting_dir, mol_name, extension='pdb')
copyfile(pdb_src_path, pdb_path)

# COPY MONOMER FILE WITH ASSIGNED TERMINAL GROUPS
mono_src_path = assemble_path(mono_src_dir, mol_name, extension='json')
assert(mono_src_path.exists())
mono_path = assemble_path(starting_dir, mol_name, extension='json')

monomers = MonomerGroup.from_file(mono_src_path)
monomers.term_orient = term_orients[mol_name]
monomers.to_file(mono_path)

## 1) Assigning chemistry, generating library charges

In [4]:
from openff.toolkit import Topology

from polysaccharide2.openfftools import TKREGS
from polysaccharide2.openfftools import topology
from polysaccharide2.residues.partition import partition


param_dir = working_dir / 'param_structs'
param_dir.mkdir(exist_ok=True)

# generate substructure cover and partition
monogrp = MonomerGroup.from_file(mono_path)
offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit'])
was_partitioned = partition(offtop)
assert(was_partitioned)

# assign properties to Molecule
offmol = topology.get_largest_offmol(offtop)
offmol.name = mol_name
offmol.properties['solvent'] = None
offmol.properties['charge_method'] = None

# save partitioned Topology
mol_path = param_dir / f'{mol_name}.sdf'
topology.topology_to_sdf(mol_path, offtop=offtop, toolkit_registry=TKREGS['The RDKit'])

## 2) Assigning charges

### 2a) RCT + generating library charges

In [5]:
from openff.toolkit import Molecule
from polysaccharide2.openfftools.pcharge import MolCharger
from polysaccharide2.genutils.decorators.functional import allow_string_paths

from polysaccharide2.monomers import MonomerGroup
from polysaccharide2.polymers import estimation, building
from polysaccharide2.polymers.exceptions import MorphologyError

from polysaccharide2.residues.rescharge.interface import LibraryCharger
from polysaccharide2.residues.rescharge.calculation import compute_residue_charges
from polysaccharide2.residues.rescharge.rctypes import ChargesByResidue


# building parameters
N : int = 150
base_charging_method : str = 'Espaloma-AM1-BCC'
delete_pdb : bool=True


@allow_string_paths
def rct_protocol(working_dir : Path, mol_name : str, monogroup : MonomerGroup, N : int, charger : MolCharger, delete_pdb : bool=True) -> tuple[ChargesByResidue, Molecule]:
    '''
    Generates library charges for a monomer group given terminal group head-tail orientations and a maximum chain length
    If delete_pdb=True, will remove the working pdb path after charges are generated
    as currently implemented, only supports monomer groups which constitute a linear homopolymer
    '''
    if not monogroup.is_linear:
        raise MorphologyError('RCT currently only supports linear homopolymers')
    
    DOP = estimation.estimate_DOP_lower(monogroup, max_chain_len=N)
    chain = building.build_linear_polymer(monogroup, DOP)

    pdb_path = assemble_path(working_dir, mol_name, extension='pdb')
    building.mbmol_to_openmm_pdb(pdb_path, chain) # output PDB file to disc 
    offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogroup.monomers, toolkit_registry=TKREGS['The RDKit']) # load custom substructures - raises error if PDB has issues
    if delete_pdb:
        pdb_path.unlink()
        
    was_partitioned = partition(offtop) 
    assert(was_partitioned)
    offmol = topology.get_largest_offmol(offtop)
    offmol.name = mol_name

    cmol = charger.charge_molecule(offmol, in_place=False)
    res_chgs = compute_residue_charges(cmol, monogroup)

    return res_chgs, cmol


# RCT loop
chgr = MolCharger.subclass_registry[base_charging_method]()
monogrp = MonomerGroup.from_file(mono_path)
lib_chgs, cmol_redux = rct_protocol(param_dir, mol_name, monogrp, N, charger=chgr, delete_pdb=delete_pdb)
mol_name_redux = f'{cmol_redux.name}_reduced'
cmol_redux.name = mol_name_redux

lib_chg_path = assemble_path(param_dir, mol_name, extension='json', postfix='residue_charges')
lib_chgs.to_file(lib_chg_path)

# save small molecule with explicit and RCT charges for benchmark
sdf_path_exact = assemble_path(param_dir, mol_name_redux, extension='sdf', postfix=base_charging_method)
topology.topology_to_sdf(sdf_path_exact, cmol_redux.to_topology())

lib_charger = LibraryCharger(lib_chgs)
rctmol_redux = lib_charger.charge_molecule(cmol_redux, in_place=False)
sdf_path_rct = assemble_path(param_dir, mol_name_redux, extension='sdf', postfix=lib_charger.CHARGING_METHOD)
topology.topology_to_sdf(sdf_path_rct, rctmol_redux.to_topology())

2023-10-11 17:39:27.361 [INFO    :        building:line 74  ] - Using pre-defined terminal group orientation {'peg_TERM2': 'head', 'peg_TERM3': 'tail'}
2023-10-11 17:39:27.363 [INFO    :        building:line 85  ] - Registering middle monomer peg (block identifier "A")
2023-10-11 17:39:27.431 [INFO    :        building:line 95  ] - Registering terminal monomer peg_TERM2 (orientation "head")
2023-10-11 17:39:27.442 [INFO    :        building:line 95  ] - Registering terminal monomer peg_TERM3 (orientation "tail")
2023-10-11 17:39:27.453 [INFO    :        building:line 102 ] - Assembling linear polymer chain with 20 monomers (149 atoms)
2023-10-11 17:39:27.517 [INFO    :        building:line 106 ] - Successfully assembled linear polymer chain with 20 monomers (149 atoms)
2023-10-11 17:39:28.681 [INFO    :         pcharge:line 34  ] - Assigning partial charges via the "Espaloma-AM1-BCC" method
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/espaloma_charge/openff_w

### 2b) Generate charged mols with chosen methods

In [6]:
from polysaccharide2.openfftools import topology
from polysaccharide2.openfftools.pcharge import MolCharger
from polysaccharide2.residues.rescharge.rctypes import ChargesByResidue


charge_methods : list[str] = [
    'Espaloma-AM1-BCC',
    'RCT'
]
library_charges_path : Optional[Path] = lib_chg_path
chg_parent_dir = working_dir / 'sims_by_charge'
chg_parent_dir.mkdir(exist_ok=True)

for charge_method in charge_methods:
    chg_dir = chg_parent_dir / charge_method
    chg_dir.mkdir(exist_ok=True)

    offtop = topology.topology_from_sdf(mol_path)
    offmol = topology.get_largest_offmol(offtop)
    mol_name = offmol.name

    ChargerType = MolCharger.subclass_registry[charge_method]
    if charge_method == 'RCT':
        if lib_chg_path is not None and lib_chg_path.exists():
            charger = ChargerType(ChargesByResidue.from_file(lib_chg_path))
        else:
            raise FileExistsError
    else:
        charger = ChargerType()

    cmol = charger.charge_molecule(offmol, in_place=False)
    charged_top_path = assemble_path(chg_dir, mol_name, extension='sdf', postfix=charge_method)
    topology.topology_to_sdf(charged_top_path, cmol.to_topology())

2023-10-11 17:39:29.162 [INFO    :         pcharge:line 34  ] - Assigning partial charges via the "Espaloma-AM1-BCC" method
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'peg_modified' and SMILES '[H]C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([

## 3) Conformer anneal and solvation

In [7]:
import numpy as np
from copy import deepcopy

from openff.toolkit import ForceField
from openmm.unit import gram, nanometer, centimeter, angstrom

from polysaccharide2.genutils.fileutils.pathutils import assemble_path
from polysaccharide2.genutils.logutils.IOHandlers import MSFHandlerFlex
from polysaccharide2.genutils.unitutils import openmm_to_openff

from polysaccharide2.openmmtools.parameters import SimulationParameters
from polysaccharide2.openmmtools import execution
from polysaccharide2.openmmtools.parameters import SimulationParameters

from polysaccharide2.openfftools.omminter import openff_topology_to_openmm
from polysaccharide2.openfftools.solvation.boxvectors import VectorQuantity, BoxVectorsQuantity
from polysaccharide2.openfftools.solvation import solvents
from polysaccharide2.openfftools.solvation.packing import pack_topology_with_solvent


# input parameters
sim_dir_name = 'conf_1'
param_top_path = charged_top_path
solvent_name = 'water_TIP3P'

sim_dir = chg_dir / sim_dir_name
sim_dir.mkdir(exist_ok=True)

box_dims = 5.0 * np.ones(3) * nanometer
density = 0.997 * gram/centimeter**3
exclusion = 1.0 * nanometer
ff_name = 'openff-2.0.0'

# define parameters
solvent  = getattr(solvents, solvent_name)
sim_param_path = Path('sim_param_sets')
anneal_params = SimulationParameters.from_file(sim_param_path / 'anneal_params.json')

# anneal function
@allow_string_paths
def vacuum_anneal(working_dir : Path, offtop : Topology, anneal_params : SimulationParameters, forcefield : Union[ForceField, str, Path],
                   box_vecs : Optional[Union[VectorQuantity, BoxVectorsQuantity]]=None, step_name : str='anneal') -> Topology:
    '''Run a short vacuum simulation of a Topology with a single molecule and reassign its conformer based on the '''
    if offtop.n_molecules != 1:
        raise ValueError(f'Can only run vacuum anneal on Topology with single molecule (Topology contains {offtop.n_molecules} molecules)')
    offmol = topology.get_largest_offmol(offtop)
    mol_name = offmol.name

    # perform conformer anneal simulation in OpenMM
    ommtop, ommsys, ommpos = openff_topology_to_openmm(offtop, forcefield=forcefield, box_vecs=box_vecs)
    anneal_schedule = {step_name : anneal_params}
    anneal_history = execution.run_simulation_schedule(working_dir, anneal_schedule, ommtop, ommsys, ommpos, return_history=True)
    new_conformer = execution.get_simulation_positions(anneal_history[step_name]['simulation'])

    # update conformer with new positions
    new_mol = deepcopy(offmol)
    LOGGER.info(f'Transferring coordinates from anneal to {mol_name} conformer')
    new_mol.conformers[0] = openmm_to_openff(new_conformer.in_units_of(angstrom)) # convert to correct units in the OpenFF format

    return new_mol.to_topology()

# initialize simulation
with MSFHandlerFlex(sim_dir, proc_name='vacuum_anneal', loggers='all') as log_handler:
    offtop = topology.topology_from_sdf(param_top_path)
    offmol = topology.get_largest_offmol(offtop)
    mol_name = offmol.name

    conf_top = vacuum_anneal(sim_dir, offtop, anneal_params, forcefield=ff_name, box_vecs=box_dims)
    conf_top_path = assemble_path(sim_dir, mol_name, extension='sdf', postfix=sim_dir_name)
    topology.topology_to_sdf(conf_top_path, conf_top)
    
    solv_top = pack_topology_with_solvent(conf_top, solvent, box_vecs=box_dims, density=density, exclusion=exclusion)
    solv_top_path = assemble_path(sim_dir, conf_top_path.stem, extension='sdf', postfix=f'solv_{solvent.name}')
    topology.topology_to_sdf(solv_top_path, solv_top)

2023-10-11 17:39:30.613 [INFO    :      parameters:line 2993] - Attempting to up-convert Electrostatics section from 0.3 to 0.4
2023-10-11 17:39:30.614 [INFO    :      parameters:line 3003] - Successfully up-converted Electrostatics section from 0.3 to 0.4. `method="PME"` is now split into `periodic_potential="Ewald3D-ConductingBoundary"`, `nonperiodic_potential="Coulomb"`, and `exception_potential="Coulomb"`.
2023-10-11 17:39:31.550 [INFO    :       execution:line 38  ] - Initializing simulation 1/1 ("anneal")
2023-10-11 17:39:31.553 [INFO    :          thermo:line 83  ] - Created LangevinMiddleIntegrator for NVT (Canonical) ensemble
2023-10-11 17:39:31.740 [INFO    :       reporters:line 127 ] - Prepared DCDReporter which reports to workflow_test_peg_modified/sims_by_charge/RCT/conf_1/anneal/anneal_trajectory.dcd
2023-10-11 17:39:31.740 [INFO    :       reporters:line 127 ] - Prepared CheckpointReporter which reports to workflow_test_peg_modified/sims_by_charge/RCT/conf_1/anneal/anne

## 4) Simulation schedule

In [ ]:
from openmm.unit import nanometer

from polysaccharide2.genutils.logutils.IOHandlers import MSFHandlerFlex
from polysaccharide2.openmmtools import parameters, execution
from polysaccharide2.openmmtools.parameters import SimulationParameters
from polysaccharide2.openfftools import topology
from polysaccharide2.openfftools.omminter import openff_topology_to_openmm


# input parameters
sim_top_path = solv_top_path

box_dims = 5.0 * np.ones(3) * nanometer
ff_name = 'openff-2.0.0'

# define paths
prod_schedule = {
    'equilibration' : parameters.SimulationParameters.from_file(sim_param_path / 'equilibration_params.json'),
    'production' : parameters.SimulationParameters.from_file(sim_param_path / 'production_lite_params.json'),
}

# initialize simulation
with MSFHandlerFlex(sim_dir, proc_name='sim_schedule', loggers='all') as log_handler:
    offtop = topology.topology_from_sdf(solv_top_path)
    ommtop, ommsys, ommpos = openff_topology_to_openmm(offtop, forcefield=ff_name, box_vecs=box_dims)
    history = execution.run_simulation_schedule(sim_dir, prod_schedule, ommtop, ommsys, ommpos, return_history=True)

## 5) Analysis

In [ ]:
sim_paths_path : Path # -> SimulationPaths

import mdtraj
import matplotlib.pyplot as plt

from polysaccharide2.openmmtools.serialization import SimulationPaths
from polysaccharide2.analysis import mdtrajutils


sim_paths = history['production']['paths']
traj = mdtraj.load(sim_paths.trajectory_path, top=sim_paths.topology_path)
pair_dict = {
    'chain O - water O' : traj.top.select_pairs('not water and element O', 'water and element O')
}
if 'N' in mdtrajutils.unique_elem_types(traj):
    pair_dict['chain N - water O'] = traj.top.select_pairs('not water and element N', 'water and element O')


rdf_dframe = mdtrajutils.acquire_rdfs(traj, pair_dict=pair_dict, min_rad=0.2, max_rad=1.2, rad_unit=nanometer)
radii, rdf = mdtrajutils.rdfs_to_plot_data(rdf_dframe)

plt.plot(radii, rdf)